<a href="https://colab.research.google.com/github/SanShiDu/noAKF/blob/main/HW2__ClinicalBERT_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
# Use a pipeline as a high-level helper
from transformers import pipeline


In [ ]:
file_path_note = '/content/drive/MyDrive/ColabData/HW2/Dynamic_noteP.csv'
note_DF = pd.read_csv(file_path_note)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModel.from_pretrained("medicalai/ClinicalBERT")

# use the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
def process_text(text):
    # crop tokens exceeds 256 tokens
    inputs = tokenizer(text, truncation=True, max_length=256, return_tensors="pt").to(device)

    input_ids = inputs["input_ids"]
    with torch.no_grad():
        features = model(input_ids)[0].detach().cpu().numpy()

    # take mean
    pooled_features = np.mean(features, axis=1)
    pooled_features_df = pd.DataFrame(pooled_features)

    return pooled_features_df

In [ ]:
features_list = []
batch_size = 32  #batch size
for i in range(0, len(note_DF["text"]), batch_size):
    batch_texts = note_DF["text"][i:i+batch_size].tolist()
    batch_features = [process_text(text) for text in batch_texts]
    features_list.extend(batch_features)

features_df = pd.concat(features_list, ignore_index=True)

In [ ]:
note_DF_output = pd.concat([note_DF.reset_index(drop=True), features_df.reset_index(drop=True)], axis=1)
note_DF_output.to_csv('/content/drive/MyDrive/ColabData/HW2/note_features.csv', index=False)